In [4]:
import cv2
import mediapipe as mp
import numpy as np
import time
import tensorflow as tf
import keras
import json


#Loading mediapipe model 
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [5]:
def get_landmarks(image_rgb, pose_model): 

    '''
    Get landmarks from a RGB image with a mediapipe model (already loaded)
    Return: a numpy array (33,4) or None if landmarks are not found 

    Arguments : 
        image_rgb : a RGB image (numpy)
        pose_model : mediapipe model 
    '''

    results = pose_model.process(image_rgb)
    
    if results.pose_landmarks:

        pose_np = np.array([[lm.x, lm.y, lm.z, lm.visibility] for lm in results.pose_landmarks.landmark])
        return pose_np

    return None

def mmss(seconds) :

    '''Convert a total number of seconds into a formatted "MM:SS" string.

        Arguments:
            seconds: Total seconds to convert (float). 
                    Negative values are handled as 0.

        Return: A string formatted as "MM:SS" (e.g., "02:05")'''

    s = max(0, int(seconds))
    return f"{s//60:02d}:{s%60:02d}"

def draw_rounded_rect(img, pt1, pt2, radius=22, color=(255, 200, 230), thickness=-1):

    ''' Display a rectangle on a image with rounding borders 

    Arguments: 
        img: image numpy 
        pt1 , pt2 : tuple of coordonates , for example (x,y) 
        radius= int, defaut is 22
        color= tuple of RGB colors, default is (255, 200, 230) (white)
        thickness= default is  -1
    '''
    
    x1, y1 = pt1
    x2, y2 = pt2
    radius = int(max(0, min(radius, abs(x2-x1)//2, abs(y2-y1)//2)))
    if thickness < 0:
        cv2.rectangle(img, (x1+radius, y1), (x2-radius, y2), color, -1)
        cv2.rectangle(img, (x1, y1+radius), (x2, y2-radius), color, -1)
        cv2.circle(img, (x1+radius, y1+radius), radius, color, -1)
        cv2.circle(img, (x2-radius, y1+radius), radius, color, -1)
        cv2.circle(img, (x1+radius, y2-radius), radius, color, -1)
        cv2.circle(img, (x2-radius, y2-radius), radius, color, -1)
    else:
        cv2.rectangle(img, pt1, pt2, color, thickness)

def put_fit_text(img, text, org, max_width, font=cv2.FONT_HERSHEY_SIMPLEX, base_scale=1.0, thickness=2, color=(60,30,60)):

    '''
    Draw text on an image and automatically scale it down if it exceeds a maximum width.

    Arguments:
        img: image numpy
        text: string to display
        org: tuple (x, y) for text position
        max_width: maximum width in pixels allowed for the text
        font: OpenCV font type
        base_scale: initial font scale
        thickness: line thickness
        color: tuple of BGR colors
    '''

    (tw, th), _ = cv2.getTextSize(text, font, base_scale, thickness)
    scale = base_scale if tw <= max_width else base_scale * (max_width / tw)
    cv2.putText(img, text, org, font, scale, color, thickness, cv2.LINE_AA)

def draw_session_card(frame, session, idx, paused=False, phase="HOLD"):

    '''
    Display the yoga session information card (UI) on the frame.

    Arguments:
        frame: image numpy
        session: list of dictionaries containing exercise data
        idx: current exercise index
        paused: boolean to indicate if the session is on pause
        phase: current phase string (e.g., "HOLD", "PREP")
    '''

    x, y = 15, 15
    box_w, box_h = 220, 70
    draw_rounded_rect(frame, (x, y), (x+box_w, y+box_h), radius=12, color=(255, 245, 250), thickness=-1)
    
    cv2.putText(frame, "YOGA", (x+12, y+22), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (60,30,60), 1)
    cv2.putText(frame, f"{idx+1}/{len(session)}", (x+box_w-45, y+22), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (60,30,60), 1)
    
    posture_name = session[idx]["name"].upper()
    put_fit_text(frame, posture_name, (x+12, y+55), max_width=box_w-25, base_scale=0.8)
    
    status_text = "PAUSED" if paused else f"STATUS: {phase}"
    cv2.putText(frame, status_text, (x+15, y+85), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (100, 0, 100), 1)

def draw_progress_bar_bottom(frame, progress, right_text=""):

    '''
    Draw a horizontal progress bar at the bottom of the screen.

    Arguments:
        frame: image numpy
        progress: float between 0 and 1 representing the percentage
        right_text: optional string to display inside the bar
    '''

    H, W = frame.shape[:2]
    margin, bar_h = 25, 20
    x, y, w = margin, H - 45, W - 2 * margin
    cv2.rectangle(frame, (x, y), (x+w, y+bar_h), (240, 230, 240), -1) # BG
    cv2.rectangle(frame, (x, y), (x+int(w*progress), y+bar_h), (80, 0, 80), -1) # Fill
    cv2.rectangle(frame, (x, y), (x+w, y+bar_h), (60, 30, 60), 1) # Border
    if right_text:
        cv2.putText(frame, right_text, (x+3, y+15), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (220, 0, 220), 1)

def draw_controls(image, is_paused):

    '''
    Display keyboard controls and a pause overlay if the session is paused.

    Arguments:
        image: image numpy
        is_paused: boolean state of the application
    '''


    h, w, _ = image.shape
    text = "P:PAUSE  N:NEXT  Q:QUIT"
    font = cv2.FONT_HERSHEY_SIMPLEX
    scale = 0.4
    thickness = 1
    
    (text_w, text_h), _ = cv2.getTextSize(text, font, scale, thickness)
    
    margin = 15
    rect_x1, rect_y1 = w - text_w - 20, margin
    rect_x2, rect_y2 = w - margin, margin + text_h + 12
    
    cv2.rectangle(image, (rect_x1, rect_y1), (rect_x2, rect_y2), (0, 0, 0), -1)
    cv2.putText(image, text, (rect_x1 + 5, rect_y2 - 6), font, scale, (255, 255, 255), thickness)

    if is_paused:
        overlay = image.copy()
        cv2.rectangle(overlay, (0, 0), (w, h), (0, 0, 0), -1)
        cv2.addWeighted(overlay, 0.3, image, 0.7, 0, image)
        cv2.putText(image, "PAUSED", (w//2 - 60, h//2), cv2.FONT_HERSHEY_DUPLEX, 1.2, (0, 0, 255), 3)

def draw_end_screen(image):

    '''
    Display a full-screen summary overlay when the yoga session is finished.

    Arguments:
        image: image numpy
    '''

    h, w, _ = image.shape
    overlay = image.copy()
    cv2.rectangle(overlay, (0, 0), (w, h), (255, 240, 250), -1)
    cv2.addWeighted(overlay, 0.8, image, 0.2, 0, image)
    
    cv2.putText(image, "END !", (w//2 - 100, h//2 - 60), cv2.FONT_HERSHEY_DUPLEX, 2.5, (60, 30, 60), 5)
    cv2.putText(image, "FINISHED ! ", (w//2 - 150, h//2 + 20), cv2.FONT_HERSHEY_DUPLEX, 1.5, (60, 30, 60), 3)
    cv2.putText(image, "BRAVO !", (w//2 - 110, h//2 + 90), cv2.FONT_HERSHEY_DUPLEX, 1.2, (100, 0, 100), 2)
    cv2.putText(image, "Put 'Q' to quit", (w//2 - 115, h - 50), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (50, 50, 50), 1)

def change_duration(session, posture, duration):
    '''
    Changes the duration of a specific posture in the session list.
    Returns True if found and changed, False otherwise.
    '''
    
    found = False
    for item in session:
        if item['name'].strip().lower() == posture.strip().lower():
            item['duration'] = max(0, int(duration))
            found = True
            
    if not found:
        print(f"Warning: Posture '{posture}' not found in session.")
        
    return found

def delete_pose(session, posture):
    '''
    Delete a pose in the session.
    Return the updated session.
    '''

    target = posture.strip().lower()
    initial_count = len(session)

    session[:] = [p for p in session if p['name'].strip().lower() != target]
    
    if len(session) == initial_count:
        print(f"Warning: Posture '{posture}' not found in session.")
        
    return session

In [ ]:
with open('yoga_config.json', 'r') as f:
    config = json.load(f)

labels = config['labels']
model = tf.keras.models.load_model(config['model_path'])

cfg = json.load(open("yoga_config.json", "r", encoding="utf-8"))
logos = np.load(cfg["logos_npz_path"])  # dict-like
pose_logo = {}
for name in logos: 
    pose_logo[name] = logos[name]    
    
session = [{'name' :i , "duration" : 15} for i in config['labels']]

# change_duration(session, posture, duration) 
session = delete_pose(session, 'cobra')

mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils


cap = cv2.VideoCapture(0)
cam_w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
cam_h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
zoom = 1.2
cv2.namedWindow("Yoga postures", cv2.WINDOW_NORMAL)
cv2.resizeWindow("Yoga postures", int(cam_w*zoom), int(cam_h*zoom))


x = 250
y = 10


idx = 0
paused = False
start_time = None
elapsed_time = 0
is_preparing = True
is_finished = False 
prep_duration = 10 
prep_start_time = time.time()

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose_model:
    while cap.isOpened():
        ret, frame = cap.read()

        if not ret: break

        frame = cv2.flip(frame, 1)

        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose_model.process(image_rgb)
        landmarks_np = get_landmarks(image_rgb, pose_model)
        image_bgr = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2BGR)


        phase = "WAITING..."
        progress = 0

        if is_finished:
            draw_end_screen(image_bgr)
            progress = 1 

        elif is_preparing:
            
            current_pose_name = session[idx]['name']
            
            if current_pose_name in pose_logo:
                img_to_show = pose_logo[current_pose_name]
                h_img, w_img, _ = img_to_show.shape
                image_bgr[y : y+h_img, x : x+w_img] = img_to_show


            elapsed_prep = time.time() - prep_start_time
            countdown = int(prep_duration - elapsed_prep)
            
            cv2.putText(image_bgr, f"GOT READY : {session[idx]['name']}", (cam_w//2-180, cam_h//2-40), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
            cv2.putText(image_bgr, str(countdown if countdown > 0 else "GO!"), (cam_w//2-30, cam_h//2+60), 
                        cv2.FONT_HERSHEY_DUPLEX, 2, (0, 255, 0), 4)
            
            if elapsed_prep >= prep_duration:
                is_preparing = False
                start_time = None 
                elapsed_time = 0

        else:

            is_pose_correct = False

            if not paused:


                # # Modele prediction posture 
                if landmarks_np is not None:
                    input_data = landmarks_np.flatten().reshape(1, -1) #format (33,4) --> (1, 132)
                    
                    prediction = model.predict(input_data, verbose=0)
                    predicted_idx = np.argmax(prediction)
                    confidence = prediction[0][predicted_idx]
                    
                    current_target_name = session[idx]["name"]
                    predicted_name = labels[predicted_idx]
                    
                if predicted_name == current_target_name and confidence > 0.8:
                    is_pose_correct = True

                #is_pose_correct = False
                #is_pose_correct = landmarks_np is not None # to test the model 
                
                if is_pose_correct:
                    phase = "HOLDING"

                    now = time.time()
                    if start_time is None: 

                        start_time = now
                    
                    delta = now - start_time
                    elapsed_time += delta  
                    start_time = now       


                    target_duration = session[idx]["duration"]
                    progress = np.clip(elapsed_time / target_duration, 0, 1)

                    if elapsed_time >= target_duration:
                        if idx < len(session) - 1:
                            idx += 1
                            is_preparing = True
                            prep_start_time = time.time()
                            start_time = None
                            elapsed_time = 0
                        else:
                            is_finished = True 
                else:
                    start_time = None

            # if results.pose_landmarks:
            #     mp_drawing.draw_landmarks(image_bgr, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
            
        draw_session_card(image_bgr, session, idx, paused=paused, phase=phase)
        remaining_time = max(0, session[idx]["duration"] - elapsed_time)
        draw_progress_bar_bottom(image_bgr, progress, right_text=mmss(remaining_time))
        draw_controls(image_bgr, paused)
        cv2.imshow('Yoga postures', image_bgr)

        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'): break
        elif key == ord('p'): paused = not paused
        elif key == ord('n'):
            if idx < len(session) - 1:
                idx += 1
                is_preparing = True
                prep_start_time = time.time()
                start_time = None
                elapsed_time = 0
            else:
                is_finished = True

    cap.release()
    cv2.destroyAllWindows()